# Textbook Problem 12.9-2

Consider the following problem:

Maximize $Z = 5x_1 - x_1^2 + 8x_2 - x_2^2 + 10x_3 - x_3^2 + 15 x_4 - x_4^2 + 20 x_5 - x_5^2$

subject to 

$ x_1 \in \{3,6,12\}, x_2 \in \{3,6\}, x_3 \in \{3,6,9,12\}, x_4 \in \{6,12\}, x_5 \in \{9,12,15,18\}$

$x_1, x_2, x_3, x_4, x_5$ must all be different 

$x_1 + x_3 + x_4 \leq 25$

**(a)** Without doing any optimization, use CP-SAT to make a list of all feasible solutions.  Your code should be easily generalizable (use an abstract approach).  

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
from ortools.sat.python import cp_model

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f'{v} = {self.Value(v)}', end = ' ')
        print()

    def solution_count(self):
        return self.__solution_count

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
sets_dict = { 'x1':[3,6,12], 'x2':[3,6], 'x3':[3,6,9,12], 'x4': [6,12], 
            'x5': [9,12,15,18]}

dvars = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(sets_dict[v]),v) for v in sets_dict.keys()]

# Creates the constraints.
model.Add(dvars[0] + dvars[2] + dvars[3] <= 25)
model.AddAllDifferent(dvars)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x for x in dvars])
status = solver.SearchForAllSolutions(model, solution_printer)

print(f'Status = {solver.StatusName(status)}')
print(f'Number of solutions found: {solution_printer.solution_count()}')

**(b)** Now use CP-SAT to solve the optimization problem.  Your code should be easily generalizable (use an abstract approach).  

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
sets_dict = { 'x1':[3,6,12], 'x2':[3,6], 'x3':[3,6,9,12], 'x4': [6,12], 
            'x5': [9,12,15,18]}

dvars = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(sets_dict[v]),v) for v in sets_dict.keys()]
xsq = [model.NewIntVar(min(sets_dict[v])**2, max(sets_dict[v])**2, f'x{i}sq') for i, v in enumerate(sets_dict.keys(),1)]

# Creates the constraints.
for i, x in enumerate(dvars,1):
    model.AddMultiplicationEquality(xsq[i-1], [x,x])

model.Add(dvars[0] + dvars[2] + dvars[3] <= 25)
model.AddAllDifferent(dvars)

# Add an objective function and a direction, need not be linear
coefs = [5,8,10,15,20]
model.Maximize( sum( (coefs[i]*dvars[i])-xsq[i] for i in range(len(dvars))) )

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(f'The maximum value of the objective function is {solver.ObjectiveValue()}')
    print()
    for x in dvars:
        print(f'{x} = {solver.Value(x)}')

# Assignment Problem

This problem is based on textbook problem 9.1-7 

The Move-It Company has two plants producing forklift trucks that then are shipped to three distribution centers. The production costs are the same at the two plants, and the cost of shipping for each truck is shown for each combination of plant and distribution center: 

<img src="./images/forklift_table.png" width=400>

A total of 60 forklift trucks are produced and shipped per week. Each plant can produce and ship any amount up to a maximum of 50 trucks per week, so there is considerable flexibility on how to divide the total production between the two plants so as to reduce shipping costs. However, each distribution center must receive exactly 20 trucks per week.

The objective of management is to determine how many forklift trucks should be produced at each plant, and then what the overall shipping pattern should be to minimize total shipping cost.

We are going to solve this two different ways:

**(a)** *Allow product splitting.*  Each distribution center can receive forklift trucks from both plants (it's possible distribution center 1 gets 10 from A and 10 from B). Solve this transportation by adapting abstract Pyomo code from Lesson 3.  Note that the total supply is greater than the total demand so you'll to include a dummy distribution center that receives the excess supply to turn this into a balanced transportation problem.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
plants = ['A', 'B']
supply = dict(zip(plants, [50, 50]))

distros = ['dist1','dist2','dist3']
demand = dict(zip(distros, [20, 20, 20]))

usc = [[600, 700, 400], [700, 800, 500]]
unit_ship_cost = {
    plants[p]: {distros[d]: usc[p][d]
                   for d in range(len(distros))}
    for p in range(len(plants))
}

from pyomo.environ import *

model = ConcreteModel()

model.transp = Var(plants, distros, domain=NonNegativeReals)

model.total_cost = Objective(expr=sum(unit_ship_cost[p][d] * model.transp[p, d]
                                      for p in plants for d in distros),
                             sense=minimize)

model.supply_ct = ConstraintList()
for p in plants:
    model.supply_ct.add(
        sum(model.transp[p, d] for d in distros) <= supply[p])

model.demand_ct = ConstraintList()
for d in distros:
    model.demand_ct.add(
        sum(model.transp[p, d] for p in plants) == demand[d])

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Minimum Total Cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))

# put amounts in dataframe for nicer display
import pandas as pd
dvars = pd.DataFrame([[model.transp[p, d]() for d in distros]
                      for p in plants],
                     index=plants,
                     columns=distros)
print("Number of truckloads to ship from each plant to each distribution:")
dvars

**(b)** *No product splitting allowed* . Each distribution center can receive forklift trucks from only one plant (to lower administrative and other hidden costs).   So one plant sends two shipments of 20 to each of two distribution centers, while the other plant sends one shipment of 20 to the remaining distribution center.

Follow the "Formulation of option 2" on page 354 to formulate this as an assignment problem, then use CP-SAT with Element and All-Different constraints to solve this assignment problem to find the minimum cost.  The minimum cost with no-splitting allowed should be larger than the minimum cost when splitting is allowed.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
# abstract version
import pandas as pd
from IPython.display import display, HTML

plants = ['A','B']
distros = ['Dist1', 'Dist2', 'Dist3']
demand = [20, 20, 20]
total_demand = 60
cost_table = [[800, 700, 400], [600, 800, 500]]

num_plants = len(cost_table)
num_distros = len(cost_table[0])

from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Variables
assign = [
    model.NewIntVar(0, num_distros - 1, plants[i])
    for i in range(num_plants)
]

max_cost = max(list(map(max, cost_table)))
cost = [model.NewIntVar(0, max_cost, f'cost{i}') for i in range(num_plants)]

# Constraints
model.AddAllDifferent(assign)

for i in range(num_plants):
    model.AddElement(assign[i], cost_table[i], cost[i])

model.Minimize(sum(cost))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(f'Lowest Possible Cost: {solver.ObjectiveValue()}')
    print()
    print('Assignments and associated costs:')
    cost_assigns = pd.DataFrame(0, index=plants, columns=distros)
    for i in range(num_plants):
        cost_assigns.iloc[i, solver.Value(assign[i])] = solver.Value(assign[i])
    display(cost_assigns)

# Different Assignment-like Problem

A college student has 7 days remaining before final examinations begin in her four courses, and she wants to allocate this study time as effectively as possible. She needs at least 1 day on each course, and she likes to concentrate on just one course each day, so she wants to allocate 1, 2, 3, or 4 days to each course. Having recently taken an OR course, she decides to use dynamic programming to make these allocations to maximize the total grade points to be obtained from the four courses. She estimates that the alternative allocations for each course would yield the number of grade points shown in the following table:

<img src="./images/grades_table.png" width=400>

Use a CP-SAT constraint programming approach with the Element constraint and other appropriate constraints to maximize the total grade points.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [30]:
# abstract version
from ortools.sat.python import cp_model
import pandas as pd

y_axis = ['1-day', '2-day', '3-day', '4-day']
x_axis = ['1c', '2c', '3c', '4c']
possible_grades = [[3,5,6,7],[5,5,6,9],[2,4,7,8],[6,7,9,9]]
courses = 4
study_bounds = [[1,4]]*courses

# Create the model.
model = cp_model.CpModel()

days_per_course = [
    model.NewIntVar(study_bounds[i][0], study_bounds[i][1], f'course{i}') 
    for i in range(courses)
]

grades = [
    model.NewIntVar(min(possible_grades[i]), max(possible_grades[i]), f'grades{i}') 
    for i in range(courses)
]

# Creates the constraints.
model.Add(sum(days_per_course) == 7)

for i in range(courses):
    model.AddElement(days_per_course[i], possible_grades[i], grades[i])
    
# Add an objective function and a direction, need not be linear
model.Maximize(sum(grades))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    for i in range(courses):
        print(f'course {i+1} = {solver.Value(days_per_course[i])} day')

Maximum of objective function: 28

course 1 = 1 day
course 2 = 3 day
course 3 = 2 day
course 4 = 1 day


# Scheduling

Use the CP-SAT approach to scheduling shown in the lesson to find a schedule of minimum length for the Reliable Construction Company project with activities A-N shown in the table below:

<img src="./images/reliable_table.png" width=600>

This problem is discussed in a supplemental textbook chapter which we've included in the folder with this notebook if you want to know more about it.

Display both the minimized length and the optimal schedule in both text and with a Gantt chart.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [37]:
import numpy as np

task_duration_dict = {
    'excavate': 2,
    'lay_foundation': 4,
    'rough_wall': 10,
    'roof': 6,
    'exterior_plumbing': 4,
    'interior_plumbing': 5,
    'exterior_siding': 7,
    'exterior_painting': 9,
    'electrical_work': 7,
    'wallboard': 8,
    'flooring': 4,
    'interior_painting': 5,
    'exterior_fixtures': 2,
    'interior_fixtures': 6
}
task_names = list(task_duration_dict.keys())
num_tasks = len(task_names)
durations = list(task_duration_dict.values())

# for each task we have a list of tasks that must go after
# task:['these','tasks','after']
precedence_dict = {
    'lay_foundation': ['excavate'],
    'rough_wall': ['lay_foundation'],
    'roof': ['rough_wall'],
    'exterior_plumbing': ['rough_wall'],
    'interior_plumbing': ['exterior_plumbing'],
    'exterior_siding': ['roof'],
    'exterior_painting': ['exterior_plumbing','exterior_siding'],
    'electrical_work': ['rough_wall'],
    'wallboard': ['electrical_work','interior_plumbing'],
    'flooring': ['wallboard'],
    'interior_painting': ['wallboard'],
    'exterior_fixtures': ['exterior_painting'],
    'interior_fixtures': ['flooring','interior_painting']
}

task_name_to_number_dict = dict(zip(task_names, np.arange(0, num_tasks)))

horizon = sum(task_duration_dict.values())

from ortools.sat.python import cp_model
model = cp_model.CpModel()

start_vars = [
    model.NewIntVar(0, horizon, name=f'start_{t}') for t in task_names
]
end_vars = [model.NewIntVar(0, horizon, name=f'end_{t}') for t in task_names]

# the `NewIntervalVar` are both variables and constraints, the internally enforce that start + duration = end
intervals = [
    model.NewIntervalVar(start_vars[i],
                         durations[i],
                         end_vars[i],
                         name=f'interval_{task_names[i]}')
    for i in range(num_tasks)
]

# precedence constraints
for before in list(precedence_dict.keys()):
    for after in precedence_dict[before]:
        before_index = task_name_to_number_dict[before]
        after_index = task_name_to_number_dict[after]
        model.Add(end_vars[before_index] <= start_vars[after_index])

obj_var = model.NewIntVar(0, horizon, 'largest_end_time')
model.AddMaxEquality(obj_var, end_vars)
model.Minimize(obj_var)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
for i in range(num_tasks):
    print(
        f'{task_names[i]} start at {solver.Value(start_vars[i])} and end at {solver.Value(end_vars[i])}'
    )

Optimal Schedule Length: 44.0
excavate start at 42 and end at 44
lay_foundation start at 38 and end at 42
rough_wall start at 28 and end at 38
roof start at 18 and end at 24
exterior_plumbing start at 24 and end at 28
interior_plumbing start at 19 and end at 24
exterior_siding start at 11 and end at 18
exterior_painting start at 2 and end at 11
electrical_work start at 19 and end at 26
wallboard start at 11 and end at 19
flooring start at 6 and end at 10
interior_painting start at 6 and end at 11
exterior_fixtures start at 0 and end at 2
interior_fixtures start at 0 and end at 6
